<a href="https://colab.research.google.com/github/GreenD93/Databiz_AI/blob/main/img/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
print("📂 Connecting to Google Drive...")
drive.mount('/content/drive')

📂 Connecting to Google Drive...
Mounted at /content/drive


In [ ]:
# game icon url
# base model : https://huggingface.co/runwayml/stable-diffusion-v1-5
# https://civitai.com/models/284567/game-icon-institute3d
# https://civitai.com/api/download/models/320184
# download original modelsGame Icon Institute3d
# base models
# !wget /content https://civitai.com/api/download/models/320184 --content-disposition

In [ ]:
import os
import toml
from time import time

# model config
## more details : https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb
COLAB = True # low ram

root_dir = "/content" if COLAB else "~/Loras"
repo_dir = os.path.join(root_dir, "kohya-trainer")

main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
log_folder    = os.path.join(main_dir, "_logs")
project_name  = "kbank_icon" # <-- 폴더 이름 변경

config_folder = os.path.join(main_dir, project_name)
images_folder = os.path.join(main_dir, project_name, "dataset")
output_folder = os.path.join(main_dir, project_name, "output")

# base model
model_file = os.path.join(config_folder, "gameIconInstitute3d_v10.safetensors")

# paths
config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")


In [ ]:
COLAB = True # low ram
XFORMERS = True
SOURCE = "https://github.com/kohya-ss/sd-scripts"
COMMIT = "9a67e0df390033a89f17e70df5131393692c2a55"
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True

def install_dependencies():
  os.chdir(root_dir)
  !git clone {SOURCE} {repo_dir}
  os.chdir(repo_dir)
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_network_wrapper.py -q -O train_network_wrapper.py
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/dracula.py -q -O dracula.py

  !apt -y update -qq
  !apt -y install aria2 -qq
  !pip install accelerate==0.15.0 diffusers==0.10.2 transformers==4.26.0 bitsandbytes==0.41.3.post2 opencv-python==4.8.0.76 tensorflow torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 jax==0.4.23 jaxlib==0.4.23
  !pip install toml==0.10.2 ftfy==6.1.1 einops==0.6.0 timm==0.6.12 fairscale==0.4.13 albumentations==1.3.1 voluptuous==0.13.1 requests==2.31.0 pytorch-lightning==1.9.0
  !pip install safetensors lion_pytorch==0.0.6 dadaptation==3.1 prodigyopt==1.0 pygments
  !pip install .
  if XFORMERS:
    !pip install xformers==0.0.22.post7

  # patch kohya for minor stuff
  if COLAB:
    !sed -i "s@cpu@cuda@" library/model_util.py # low ram
  if LOAD_TRUNCATED_IMAGES:
    !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  if BETTER_EPOCH_NAMES:
    !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
    !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"

install_dependencies()

Cloning into '/content/kohya-trainer'...
remote: Enumerating objects: 6424, done.
remote: Counting objects: 100% (3220/3220), done.
remote: Compressing objects: 100% (584/584), done.
remote: Total 6424 (delta 2946), reused 2748 (delta 2634), pack-reused 3204
Receiving objects: 100% (6424/6424), 9.49 MiB | 23.07 MiB/s, done.
Resolving deltas: 100% (4588/4588), done.
HEAD is now at 9a67e0d Merge pull request #610 from lubobill1990/patch-1
46 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 46 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.

In [ ]:
# https://civitai.com/articles/4/make-your-own-loras-easy-and-free

config_dict = {
  'additional_network_arguments': {
    'unet_lr': 0.0005,
    'text_encoder_lr': 0.0001,
    'network_dim': 16,
    'network_alpha': 8,
    'network_module': 'networks.lora'
  },
  'optimizer_arguments': {
    'learning_rate': 0.0005,
    'lr_scheduler': 'cosine_with_restarts',
    'lr_scheduler_num_cycles': 3,
    'lr_warmup_steps': 272,
    'optimizer_type': 'AdamW8bit'
  },
  'training_arguments': {
    'max_train_epochs': 10,
    'save_every_n_epochs': 1,
    'save_last_n_epochs': 10,
    'train_batch_size': 2,
    'clip_skip': 2,
    'min_snr_gamma': 5.0,
    'weighted_captions': False,
    'seed': 42,
    'max_token_length': 225,
    'xformers': True,
    'lowram': True,
    'max_data_loader_n_workers': 8,
    'persistent_data_loader_workers': True,
    'save_precision': 'fp16',
    'mixed_precision': 'fp16',
    'output_dir': output_folder,
    'logging_dir': log_folder,
    'output_name': project_name,
    'log_prefix': project_name
  },
  'model_arguments': {
    'pretrained_model_name_or_path': model_file,
    'v2': False,
    "v_parameterization": None
  },
  'saving_arguments': {
    'save_model_as': 'safetensors'
  },
  'dreambooth_arguments': {
    'prior_loss_weight': 1.0
  },
  'dataset_arguments': {
    'cache_latents': True
  }
}

# 10 images × 10 repeats × 20 epochs ÷ 2 batch size = 1000 steps
# 20 images × 10 repeats × 10 epochs ÷ 2 batch size = 1000 steps
# 100 images × 3 repeats × 10 epochs ÷ 2 batch size = 1500 steps
# 400 images × 1 repeat × 10 epochs ÷ 2 batch size = 2000 steps
# 1000 images × 1 repeat × 10 epochs ÷ 3 batch size = 3300 steps

dataset_config_dict = {
  'general': {
    'resolution': 512,
    'shuffle_caption': True,
    'keep_tokens': 1,
    'flip_aug': False,
    'caption_extension': '.txt',
    'enable_bucket': True,
    'bucket_reso_steps': 64,
    'bucket_no_upscale': False,
    'min_bucket_reso': 256,
    'max_bucket_reso': 1024
  },
  'datasets': [
    {
      'subsets': [
        {
          'num_repeats': 3,
          'image_dir': images_folder,
          'class_tokens': None
        }
      ]
    }
  ]
}

# write config dict
with open(config_file, "w") as f:
    f.write(toml.dumps(config_dict))

# write dataset_config dict
with open(dataset_config_file, "w") as f:
    f.write(toml.dumps(dataset_config_dict))

In [ ]:
!accelerate launch --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
Loading settings from /content/drive/MyDrive/Loras/kbank_icon/training_config.toml...
/content/drive/MyDrive/Loras/kbank_icon/training_config
prepare tokenizer
vocab.json: 100% 961k/961k [00:00<00:00, 29.4MB/s]
merges.txt: 100% 525k/525k [00:00<00:00, 2.72MB/s]
special_tokens_map.json: 100% 389/389 [00:00<00:00, 2.78MB/s]
tokenizer_config.json: 100% 905/905 [00:00<00:00, 7.16MB/s]
update token length: 225
Loading dataset config from /content/drive/MyDrive/Loras/kbank_icon/dataset_config.toml
prepare images.
found directory /content/drive/MyDrive/Loras/kbank_icon/dataset contains 107 image files
321 train images with repeating.
0 reg images.
no regularization images / 正則化画像が見つかりませんでした
[Dataset 0]
  batch_size: 2
  resolution: (512, 512)
  enable_bucket: True
  min_bucket_reso: 256
  max_bucket_reso: 1024
  bucket_reso_steps: 64
  bucket_no_upscale: False

  [Subset 0 of Dataset